In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.autograd import Variable

In [2]:
a=np.load("train_data.npy").reshape(51146,2688)
train_data=a[~np.isnan(a).any(axis=1),:].reshape(35225, 42, 8, 8)
b=np.load("train_label.npy").reshape(-1,1)
train_label=b[~np.isnan(a).any(axis=1),:].reshape(-1,)
a=np.load("valid_data.npy").reshape(5371,2688)
valid_data=a[~np.isnan(a).any(axis=1),:].reshape(4577, 42, 8, 8)
b=np.load("valid_label.npy").reshape(-1,1)
valid_label=b[~np.isnan(a).any(axis=1),:].reshape(-1,)

In [3]:
train_data=train_data.reshape([train_data.shape[0],train_data.shape[1],64])
valid_data=valid_data.reshape([valid_data.shape[0],valid_data.shape[1],64])

In [4]:
train_data=np.transpose(train_data,(0,2,1))
valid_data=np.transpose(valid_data,(0,2,1))

In [5]:
o=np.array([[[1,2,3],[2,3,4]],[[5,4,9],[8,2,9]]])
o,o.shape

(array([[[1, 2, 3],
         [2, 3, 4]],
 
        [[5, 4, 9],
         [8, 2, 9]]]), (2, 2, 3))

In [6]:
np.transpose(o,(0,2,1))

array([[[1, 2],
        [2, 3],
        [3, 4]],

       [[5, 8],
        [4, 2],
        [9, 9]]])

In [7]:
class GetLoader(Dataset):
# 初始化函数，得到数据
    def __init__(self, data_root, data_label):
        self.data = data_root
        self.label = data_label
    # index是根据batchsize划分数据后得到的索引，最后将data和对应的labels进行一起返回
    def __getitem__(self, index):
        data = self.data[index]
        labels = self.label[index]
        return data, labels
    # 该函数返回数据大小长度，目的是DataLoader方便划分，如果不知道大小，DataLoader会一执行错误
    def __len__(self):
        return len(self.data)

In [8]:
train=GetLoader(train_data,train_label)
valid=GetLoader(valid_data,valid_label)

In [9]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes,device):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)  # batch_first=True仅仅针对输入而言
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # 设置初始状态h_0与c_0的状态是初始的状态，一般设置为0，尺寸是,x.size(0)
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)

        # Forward propagate RNN
        out, (h_n, c_n) = self.lstm(x, (h0, c0))  # 送入一个初始的x值，作为输入以及(h0, c0)

        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  # output也是batch_first, 实际上h_n与c_n并不是batch_first
        return out

In [20]:
# Hyper Parameters
sequence_length = 64  # 序列长度，将图像的每一列作为一个序列
input_size = 42  # 输入数据的维度
hidden_size = 256  # 隐藏层的size
num_layers =  2 # 有多少层

num_classes = 2
batch_size = 256
num_epochs = 600
learning_rate = 0.001

device = torch.device("cuda:1")

In [11]:
lstm = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes,device=device)
lstm.to(device)

LSTM(
  (lstm): LSTM(42, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)

In [21]:
# Loss and Optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

In [13]:
valid_data.shape

(4577, 64, 42)

In [14]:
train_loader=torch.utils.data.DataLoader(dataset=train,batch_size=batch_size,shuffle=True,num_workers=1)
valid_loader=torch.utils.data.DataLoader(dataset=valid,batch_size=4577,shuffle=False,num_workers=1)

In [22]:
#训练过程
total_step=0
for epoch in range(num_epochs):
    for data in train_loader:
        images, labels = data
        images=images.type(torch.FloatTensor)
        labels=labels.type(torch.FloatTensor)
        images=images.to(device)
        labels=labels.to(device)
        #forward pass
        outputs=lstm(images)
        loss=loss_function(outputs,labels.long())
        
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_step+=1
        if (total_step)%10==0:#each 10 iterations is one epoch
            print("Epoch [{}/{}],step[{}] Loss:{:.4f}".format(epoch+1,num_epochs,total_step,loss.item()))

Epoch [1/600],step[10] Loss:0.3155
Epoch [1/600],step[20] Loss:0.3773
Epoch [1/600],step[30] Loss:0.3203
Epoch [1/600],step[40] Loss:0.2857
Epoch [1/600],step[50] Loss:0.2945
Epoch [1/600],step[60] Loss:0.3774
Epoch [1/600],step[70] Loss:0.3627
Epoch [1/600],step[80] Loss:0.3271
Epoch [1/600],step[90] Loss:0.3078
Epoch [1/600],step[100] Loss:0.3071
Epoch [1/600],step[110] Loss:0.2648
Epoch [1/600],step[120] Loss:0.2784
Epoch [1/600],step[130] Loss:0.3398
Epoch [2/600],step[140] Loss:0.2915
Epoch [2/600],step[150] Loss:0.2750
Epoch [2/600],step[160] Loss:0.2929
Epoch [2/600],step[170] Loss:0.2628
Epoch [2/600],step[180] Loss:0.3125
Epoch [2/600],step[190] Loss:0.2591
Epoch [2/600],step[200] Loss:0.2868
Epoch [2/600],step[210] Loss:0.2698
Epoch [2/600],step[220] Loss:0.2703
Epoch [2/600],step[230] Loss:0.3227
Epoch [2/600],step[240] Loss:0.3654
Epoch [2/600],step[250] Loss:0.2934
Epoch [2/600],step[260] Loss:0.3121
Epoch [2/600],step[270] Loss:0.3054
Epoch [3/600],step[280] Loss:0.2603
E

In [23]:
#计算做多的precision
with torch.no_grad():
    correct=0
    total=0
    for images, labels in valid_loader:
        images=images.type(torch.FloatTensor)
        labels=labels.type(torch.FloatTensor)
        images=images.to(device)
        labels=labels.to(device)
        outputs=lstm(images)
        _,predicted=torch.max(outputs.data,1)
        total+=sum(predicted)
        correct+=(sum(predicted*labels))
print('precision of the model on the test data: {}%'.format(100*correct/total))
#200epoch training: accuracy of the model on the test images: 52.56718374481101%
#300epoch training：accuracy of the model on the test images: 53.50666375355036%
#400epoch training: accuracy of the model on the test images: 52.676425606292334%
#600epoch training with decreasing learning rate:accuracy of the model on the test images: 54.2713567839196%,
#precision of the model on the test data: 48.69131851196289%
#700epoch training: accuracy of the model on the test images: 52.938606073847495%
#800epoch training: accuracy of the model on the test images: 53.92178282717938%,precision of the model on the test data: 50.742164611816406%

precision of the model on the test data: 48.69131851196289%


In [17]:
np.array(sum(predicted*labels).cpu()),sum(np.array(predicted.cpu()))

(array(995., dtype=float32), 1932)

In [18]:
sum(predicted)/sum(labels)

tensor(0.8974, device='cuda:1')